In [43]:
#our neural network for alphabets recognition
import numpy
import scipy.special
import matplotlib.pyplot

class neuralNetwork:
    
    def __init__(self,no_of_Inodes,no_of_Hnodes,no_of_Onodes,learning_rate):
        self.alphabets = ["A","B","C","D","E","F","G","H","I","J","K","L","M","N","O","P","Q","R","S","T","U","V","W","X","Y","Z"]
        self.inputnodes = no_of_Inodes
        self.hiddennodes = no_of_Hnodes
        self.outputnodes = no_of_Onodes
        self.lr = learning_rate
        
        self.w_input_hidden = numpy.random.normal(0.0,pow(self.inputnodes,-0.5),(self.hiddennodes,self.inputnodes))
        self.w_hidden_output = numpy.random.normal(0.0,pow(self.hiddennodes,-0.5),(self.outputnodes,self.hiddennodes))
        
        self.activation_function = lambda x: scipy.special.expit(x)
        pass
    
    # this function train our neural network
    #input_values and target_values are lists. It's not a single value
    def train(self, input_values,target_values):
        inputs = numpy.array(input_values,ndmin=2).T
        targets = numpy.array(target_values,ndmin=2).T
        
        hidden_inputs = numpy.dot(self.w_input_hidden,inputs)
        hidden_outputs = self.activation_function(hidden_inputs)
        
        final_inputs = numpy.dot(self.w_hidden_output, hidden_outputs)
        final_outputs = self.activation_function(final_inputs)
        
        output_errors = targets - final_outputs
        hidden_errors = numpy.dot(self.w_hidden_output.T,output_errors)
        
        self.w_hidden_output += self.lr * numpy.dot((output_errors * final_outputs * (1.0 - final_outputs)), 
                                                    numpy.transpose(hidden_outputs))
        
        self.w_input_hidden += self.lr * numpy.dot((hidden_errors * hidden_outputs * (1.0 - hidden_outputs)), 
                                                    numpy.transpose(inputs))
        pass
    
    #function to help us query from our neural network

    def query(self,inputs_list):

            inputs = numpy.array(inputs_list,ndmin=2).T

            signal_into_hidden_layer = numpy.dot(self.w_input_hidden,inputs)
            signal_from_hidden_layer = self.activation_function(signal_into_hidden_layer)

            signal_into_output_layer = numpy.dot(self.w_hidden_output,signal_from_hidden_layer)
            network_final_output = self.activation_function(signal_into_output_layer)

            return network_final_output
        
no_input_nodes = 784
no_hidden_nodes = 500
no_output_nodes = 26

learning_rate_value = 0.00019

#creating an instance of the neural network class
n = neuralNetwork(no_input_nodes, no_hidden_nodes,no_output_nodes,learning_rate_value)

#loading our dataset
training_file = open("lehn_dataset/alphabets_train.csv",'r')
training_list = training_file.readlines()
training_file.close()

epochs = 3

for e in range(epochs):
    for record in training_list:
        all_values = record.split(',')
        train_inputs = (numpy.asfarray(all_values[1:])/ 255.0 * 0.99) + 0.01
        train_targets = numpy.zeros(no_output_nodes) + 0.01
        train_targets[n.alphabets.index(all_values[0])] = 0.99
        n.train(train_inputs,train_targets)
        pass

print("Training Done for ", str(numpy.shape(training_list)) + " records")
        

Training Done for  (10560,) records


In [44]:
#loading our full test dataset
test_file = open("lehn_dataset/alphabets_test.csv",'r')
test_dataset = test_file.readlines()
test_file.close()

scorecard = []

#go through all the records in the test dataset
for record in test_dataset:
    all_values = record.split(',')
    label_we_want_to_predict = int(n.alphabets.index(all_values[0]))
    inputs = numpy.asfarray(all_values[1:])
    outputs = n.query(inputs)
    our_network_predicted_label = numpy.argmax(outputs)
    if(our_network_predicted_label== label_we_want_to_predict):
        scorecard.append(1)
    else:
        scorecard.append(0)
        pass
    pass

scorecard_array = numpy.asarray(scorecard)
print("Network Accuracy",scorecard_array.sum()/scorecard_array.size)

Network Accuracy 0.2962962962962963


4
